# BERT

In [ ]:
!pip install -q transformers

import os
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
input_folder_path = "./pls/Thesis_Jupyter_Final/src/input/"

def drop_missing(data):
    # Remove any rows with missing values and reset the index
    rows_before = data.shape[0] 

    data.replace('', np.nan, inplace=True)
    data = data.dropna()
    data.reset_index(drop=True, inplace=True)

    rows_after = data.shape[0]
    # Count the number of rows removed
    print(f"Number of removed rows: {rows_before - rows_after}")

    return data

def load_data(filename, process=True):
    # Load data
    data_file_path = os.path.join(input_folder_path, filename)
    df = pd.read_csv(data_file_path)

    if process:
        # Set ID as index
        df.set_index('Id', inplace=True, drop=True)
        df = drop_missing(df)

        print(df.info())
        print()
        print(f'Dataset shape: {df.shape}\n')

    return df

data_filename = "original_data.csv"
df_original = load_data(data_filename)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
MAX_LEN = 659
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05